In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# -----
from scipy import signal
from skimage.restoration import denoise_tv_chambolle
from skimage.filters import gaussian
import json
import pandas as pd
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from keras import initializers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Data and parameters

In [2]:
def channels(data):
    rgb_arrays = []
    for i, row in data.iterrows():
        
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2
        
        band_1 = denoise_tv_chambolle(band_1, weight=0.05) # chosen arbitrarily
        band_2 = denoise_tv_chambolle(band_2, weight=0.05)
        band_3 = denoise_tv_chambolle(band_3, weight=0.05)
        
        # gradient magnitude - doesn't seem to work any better than (b1 + b2)/2
#         xder = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
#         yder = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
#         arrx = signal.convolve2d(band_1, xder, mode='same')
#         arry = signal.convolve2d(band_1, yder, mode='same')
#         band_3 = np.hypot(arrx, arry) 

        rgb = np.dstack((band_1, band_2, band_3))
        rgb_arrays.append(rgb)
        
    rgb_arrays = np.array(rgb_arrays)
    for j in range(3):
        rgb_arrays[:, :, :, j] = (rgb_arrays[:, :, :, j] - np.mean(rgb_arrays[:, :, :, j]))/np.std(rgb_arrays[:, :, :, j])
    return np.array(rgb_arrays)

In [3]:
X = pd.read_json('train.json')
Y = X['is_iceberg']  # labels

X = channels(X)
Y = to_categorical(Y)

In [4]:
image_height = 75
image_width = 75
channels = 3
number_of_classes = 2

epochs = 30
batch_size = 16

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(1363, 75, 75, 3)
(241, 75, 75, 3)
(1363, 2)
(241, 2)


## Model

In [5]:
model1 = Sequential()

# 1
model1.add(Conv2D(32, (3, 3), input_shape=(image_width, image_height, channels)))
model1.add(LeakyReLU(alpha=.01))
model1.add(MaxPooling2D(pool_size=(2, 2)))

# 2
model1.add(BatchNormalization())
model1.add(Conv2D(64, (3, 3)))
model1.add(LeakyReLU(alpha=.01))
model1.add(MaxPooling2D(pool_size=(2, 2)))

# 3
model1.add(BatchNormalization())
model1.add(Conv2D(128, (3, 3)))
model1.add(LeakyReLU(alpha=.01))
model1.add(MaxPooling2D(pool_size=(2, 2)))

# 4
model1.add(BatchNormalization())
model1.add(Conv2D(256, (3, 3)))
model1.add(LeakyReLU(alpha=.01))
model1.add(MaxPooling2D(pool_size=(2, 2)))


model1.add(Flatten())

# 0
model1.add(BatchNormalization())
model1.add(Dense(512, use_bias=True, bias_initializer='zeros'))
model1.add(LeakyReLU(alpha=.01))
model1.add(Dropout(0.15))

# 1
model1.add(BatchNormalization())
model1.add(Dense(256, use_bias=True, bias_initializer='zeros'))
model1.add(LeakyReLU(alpha=.01))
model1.add(Dropout(0.15))

# 2
model1.add(BatchNormalization())
model1.add(Dense(128, use_bias=True, bias_initializer='zeros'))
model1.add(LeakyReLU(alpha=.01))
model1.add(Dropout(0.15))

# 3
model1.add(BatchNormalization())
model1.add(Dense(64, use_bias=True, bias_initializer='zeros'))
model1.add(LeakyReLU(alpha=.01))
model1.add(Dropout(0.15))

model1.add(BatchNormalization())
model1.add(Dense(number_of_classes, activation='softmax'))

In [6]:
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model1.fit(X_train, Y_train,
              batch_size=batch_size, validation_data=(X_test, Y_test),
              epochs=epochs,
                shuffle=True)

Train on 1363 samples, validate on 241 samples
Epoch 1/30
1363/1363 [==============================] - 10s - loss: 0.7165 - acc: 0.6376 - val_loss: 0.6890 - val_acc: 0.6017
Epoch 2/30
1363/1363 [==============================] - 9s - loss: 0.5731 - acc: 0.7256 - val_loss: 0.6523 - val_acc: 0.6639
Epoch 3/30
1363/1363 [==============================] - 9s - loss: 0.5127 - acc: 0.7638 - val_loss: 0.6072 - val_acc: 0.7137
Epoch 4/30
1363/1363 [==============================] - 9s - loss: 0.4794 - acc: 0.7733 - val_loss: 0.3908 - val_acc: 0.8340
Epoch 5/30
1363/1363 [==============================] - 9s - loss: 0.4345 - acc: 0.7887 - val_loss: 0.3056 - val_acc: 0.8672
Epoch 6/30
1363/1363 [==============================] - 9s - loss: 0.4014 - acc: 0.8202 - val_loss: 0.3508 - val_acc: 0.8465
Epoch 7/30
1363/1363 [==============================] - 9s - loss: 0.4450 - acc: 0.7858 - val_loss: 0.2798 - val_acc: 0.8797
Epoch 8/30
1363/1363 [==============================] - 9s - loss: 0.3591 - a

Reaches ~90% fairly quickly, then becomes a bit unstable :/

## Summary

In [69]:
Y_pred = model1.predict(X_test)
for i in range(len(Y_pred)):
    Y_pred[i, :] = np.round(Y_pred[i, :])


In [70]:
#Precision: how many 1 are true 1?
#Recall: how many 0 are true 0? 
#F1Score: balance between those two

y_comp = confusion_matrix(Y_test[:, 1], Y_pred[:, 1])
precision = y_comp[1, 1] / (y_comp[0, 1] + y_comp[1, 1])
recall = y_comp[1, 1] / (y_comp[1, 0] + y_comp[1, 1])
accuracy = accuracy_score(Y_test[:, 1], Y_pred[:, 1])
f1score = 2 * precision * recall / (precision + recall)
print("AC:%.2f PR:%.2f RE:%.2f F1:%.2f" % (accuracy, precision, recall, f1score))

AC:0.89 PR:0.82 RE:0.94 F1:0.88


In [71]:
filename = ("model_%i_%i_%i_%i.h5" % (accuracy*100, precision*100, recall*100, f1score*100))
filename2 = ("model_%i_%i_%i_%i.json" % (accuracy*100, precision*100, recall*100, f1score*100))

In [73]:
model1.save(filename)

In [72]:
model_json = model1.to_json()
with open(filename2, "w") as json_file:
    json_file.write(model_json)